In [1]:
# 17_search.ipynb

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/app')

In [2]:
from psql_helpers import read_table_async

In [3]:
from psql_models import *

In [4]:
from psql_access import get_meeting_token
from datetime import datetime
from indexing.redis_keys import RedisKeys
from indexing.worker import IndexingWorker
from redis import Redis


In [5]:
from sqlalchemy import update
from psql_helpers import get_session

async def reset_content_indexing():
    async with get_session() as session:
        await session.execute(
            update(Content).values(is_indexed=False)
        )
        await session.commit()

# Execute
#await reset_content_indexing()

In [8]:
df = await read_table_async(User)

In [9]:
df

,id,email,username,first_name,last_name,image,created_timestamp,updated_timestamp,is_indexed
0,9a8f9e71-a3d7-4831-9c8f-2ee6556e9428,test_9a8f9e71-a3d7-4831-9c8f-2ee6556e9428@exam...,test,None,None,None,2025-02-04 15:16:18.746333+00:00,2025-02-04 15:16:18.746335+00:00,False
1,aa8d9413-8d77-437f-96f1-0187bc9d412f,ryabenko.sergey@gmail.com,Sergey Ryabenko,Sergey,Ryabenko,https://lh3.googleusercontent.com/a/ACg8ocIgmE...,2025-02-04 15:16:34.858398+00:00,2025-02-04 15:16:34.858402+00:00,False
2,a6c04b0f-24d3-444a-bf21-649cd87a8be7,valeria.grigoryan199@gmail.com,Karine Stepanyan,Karine,Stepanyan,https://lh3.googleusercontent.com/a/ACg8ocJfQ4...,2025-02-04 15:16:34.935297+00:00,2025-02-04 15:16:34.935299+00:00,False
3,8b767b85-44fb-4c20-97df-7aa172242273,olganemirovskaya@gmail.com,Olga Nemirovskaya,Olga,Nemirovskaya,None,2025-02-04 15:16:34.989708+00:00,2025-02-04 15:16:34.989710+00:00,False
4,fd57278e-1cdc-4ad9-b12e-837ac39e9a55,nick@occam.fi,Nick M.,Nick,M.,None,2025-02-04 15:16:35.032834+00:00,2025-02-04 15:16:35.032836+00:00,False
5,b5651caf-b466-48a7-ac67-648a47ff3b93,Arabuli.Sh@gmail.com,Shota Arabuli,Shota,Arabuli,https://lh3.googleusercontent.com/a/ACg8ocLz4K...,2025-02-04 15:16:35.106111+00:00,2025-02-04 15:16:35.106112+00:00,False
6,b7a44b71-c622-420a-aa03-58f996f4c863,arabuli.sh@gmail.com,Shota Arabuli,Shota,Arabuli,None,2025-02-04 15:16:35.180893+00:00,2025-02-04 15:16:35.180896+00:00,False
7,ef7c085b-fdb5-4c94-b7b6-a61a3d04c210,2280905@gmail.com,Dmitriy Grankin,Dmitriy,Grankin,None,2025-02-04 15:16:35.229805+00:00,2025-02-04 15:16:35.229807+00:00,False
8,bab80c4c-74be-454e-a57a-4d41dca102f9,liza@blasterstudio.ru,Елизавета Иванова,Елизавета,Иванова,https://lh3.googleusercontent.com/a/ACg8ocKfSQ...,2025-02-04 15:16:35.277331+00:00,2025-02-04 15:16:35.277334+00:00,False
9,5c7a9e31-ba74-4c53-945c-3aaf48afca39,georgemec@gmail.com,George Bandeira,George,Bandeira,None,2025-02-04 15:16:35.398825+00:00,2025-02-04 15:16:35.398828+00:00,False


In [10]:
df = await read_table_async(Content)

In [11]:
df

,id,type,text,timestamp,parent_id,is_indexed,last_update
0,dfa86d3e-4946-414d-bd66-bd0ea472dc5c,meeting,None,2024-06-05 12:56:23.602416,None,False,2024-06-05 12:46:02.777662
1,f858cb3e-ac65-467c-8f5f-d0704abff5d3,meeting,None,2024-06-18 11:00:08.984681,None,False,2024-06-04 10:36:58.015811
2,f57553c1-1b73-47e6-97e9-ae3d5295d6d4,meeting,None,2024-06-10 15:09:50.271597,None,False,2024-06-10 14:16:30.597309
3,aa6f0a6a-60a4-4a14-a005-78a377ad6b15,meeting,None,2024-06-11 13:28:41.232765,None,False,2024-06-11 12:47:32.693585
4,167773e6-12f1-41b1-af82-8fa586fc83f7,meeting,None,2024-06-11 15:00:47.108002,None,False,2024-06-11 15:23:09.096777
...,...,...,...,...,...,...,...
5369,7a0abd55-1039-4de8-b7d9-a40b14cc992b,title,14:01,NaT,4624873e-2c5e-472c-8da2-7fb0fd798ff7,False,NaT
5370,e765512d-e223-488b-a3e0-60c3e593f243,meeting,None,2025-02-04 13:59:40.000000,None,False,2025-02-04 14:35:44.000000
5371,1634c5f9-ceed-4dfe-a187-5535fa3235a2,meeting,None,2025-02-04 15:06:37.000000,None,False,2025-02-04 15:30:34.000000
5372,152736ef-91fe-41d6-b4e1-1e7cb414e557,meeting,None,2025-02-04 14:14:09.000000,None,False,2025-02-04 14:33:52.000000


In [12]:
df[df['is_indexed']]

,id,type,text,timestamp,parent_id,is_indexed,last_update
73,947c4433-b43d-47db-8ac7-f18d29f91c01,meeting,None,2024-06-11 10:01:33.905022,None,True,2024-06-11 09:28:20.261952
74,f79d46bc-e706-4d6e-90c2-44c3ad1a0882,meeting,None,2024-06-13 21:19:04.422406,None,True,2024-06-13 21:03:47.905331
75,bc0bcb5d-5d27-437e-9be9-1d368bdcdcd3,meeting,None,2024-06-03 09:14:48.631110,None,True,2024-10-14 10:04:54.000000
76,48512bee-c060-4e74-8b05-ba19ea65ef9e,meeting,None,2024-06-11 10:44:26.759113,None,True,2024-06-11 10:32:17.850452
77,ee4e0b24-8c60-4788-9f88-b441b44cd0aa,meeting,None,2024-06-18 14:04:23.800737,None,True,2024-06-05 13:39:09.315379
...,...,...,...,...,...,...,...
5076,9bef81bc-2fc4-4037-96d2-bb42aa57db71,meeting,None,2024-12-09 10:05:25.000000,None,True,2024-12-09 10:05:51.000000
5077,d7665c6d-4025-41ff-9e44-e615b9921056,meeting,None,2024-12-11 18:03:28.000000,None,True,2024-12-11 18:03:55.000000
5078,2a6454df-80e0-4f9b-9345-9cc213605dec,meeting,None,2025-01-03 20:55:23.000000,None,True,2025-01-03 20:57:51.000000
5079,11c24ee8-5fb3-423a-b257-6cb6a2cb5661,meeting,None,2025-01-30 11:31:19.000000,None,True,2025-01-30 11:32:04.000000


In [19]:
df = await read_table_async(Content)

In [20]:
df

""


In [8]:
df['is_indexed'].value_counts()


is_indexed
False    3683
Name: count, dtype: int64

In [9]:
df['type'].value_counts()


type
meeting    3683
Name: count, dtype: int64

In [30]:
processor = ContentProcessor(
            qdrant_engine=self.qdrant_engine,
            es_engine=self.es_engine
        )

NameError: name 'self' is not defined